In [ ]:
##pip install openai

In [1]:
import openai
import pandas as pd
import time

In [ ]:
client = openai.OpenAI(api_key="Your key api here")

In [3]:
data = pd.read_csv("Preprocessed Data.csv")
data.head(5)

,Text,Link,Câu trả lời,TTHC,Ministry,Câu_hỏi_sạch,Câu_trả_lời_sạch,Tên_TTHC,Tên_cơ_quan_thực_hiện
0,Cá nhân đăng ký Bồi dưỡng nghiệp vụ đăng kiểm ...,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,"Theo quy định tại Khoản 3, Điều 8 Thông tư số ...","{'Tên': ['Cấp, cấp lại thẻ, dấu kỹ thuật đăng ...",Bộ Nông nghiệp và Phát triển nông thôn,cá nhân đăng ký bồi dưỡng nghiệp vụ đăng kiểm ...,theo quy định tại khoản 3 điều 8 thông tư số 2...,"['Cấp, cấp lại thẻ, dấu kỹ thuật đăng kiểm viê...",[['Cục Thủy sản - Bộ NN-PTNT']]
1,"Khi nào các tổ chức, cá nhân có thể gửi đề xuấ...",https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Định kỳ hàng năm theo thông báo trên trang thô...,{'Tên': ['Phê duyệt danh mục đề tài khoa học v...,Bộ Nông nghiệp và Phát triển nông thôn,khi nào các tổ chức cá nhân có thể gửi đề xuất...,định kỳ hàng năm theo thông báo trên trang thô...,['Phê duyệt danh mục đề tài khoa học và công n...,[['Vụ khoa học công nghệ và môi trường']]
2,Điều kiện công nhận đối với TBKT công nghệ sin...,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Theo Điều 6 Thông tư số 23/2010/TT-BNNPTNT ngà...,{'Tên': ['Công nhận tiến bộ kỹ thuật công nghệ...,Bộ Nông nghiệp và Phát triển nông thôn,điều kiện công nhận đối với tbkt công nghệ sin...,theo điều 6 thông tư số 23/2010/tt bnnptnt ngà...,['Công nhận tiến bộ kỹ thuật công nghệ sinh học'],[['Vụ khoa học công nghệ và môi trường']]
3,Cách thức thực hiện việc gửi đề xuất nhiệm vụ ...,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Đề xuất nhiệm vụ KHCN có thể gửi trực tiếp tại...,{'Tên': ['Phê duyệt danh mục đề tài khoa học v...,Bộ Nông nghiệp và Phát triển nông thôn,cách thức thực hiện việc gửi đề xuất nhiệm vụ ...,đề xuất nhiệm vụ khcn có thể gửi trực tiếp tại...,['Phê duyệt danh mục đề tài khoa học và công n...,[['Vụ khoa học công nghệ và môi trường']]
4,Hồ sơ đề nghị thu hồi Giấy chứng nhận cơ sở đủ...,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,"Theo quy định tại Khoản 4, Điều 54 của Nghị đị...","{'Tên': ['Cấp, cấp lại giấy chứng nhận cơ sở đ...",Bộ Nông nghiệp và Phát triển nông thôn,hồ sơ đề nghị thu hồi giấy chứng nhận cơ sở đủ...,theo quy định tại khoản 4 điều 54 của nghị địn...,"['Cấp, cấp lại giấy chứng nhận cơ sở đủ điều k...",[['Cơ quan quản lý nhà nước về thủy sản cấp tỉ...


In [4]:
data.shape[0]

6861

In [6]:
instruction_0 = """Bạn sẽ đóng vai trò là một trình tạo câu trả lời sai (hallucinated answer generator). Với một câu hỏi, câu trả lời đúng và kiến thức liên quan, mục tiêu của bạn là viết một câu trả lời nghe có vẻ hợp lý nhưng thực tế lại sai.
Bạn PHẢI tạo ra câu trả lời sai bằng cách sau:
Hiểu sai ngữ cảnh (Misinterpretation): Bạn cố gắng trả lời nhưng lại hiểu sai ngữ cảnh và ý định của câu hỏi.
Bạn PHẢI ghi rõ nhãn loại sai **trước câu trả lời** theo định dạng sau:  
**[Hiểu sai ngữ cảnh]: Câu trả lời bịa đặt...**  
Chỉ trả lời theo yêu cầu mà không nhắc lại hướng dẫn hoặc lưu ý gì khác."""

instruction_1 = """Bạn sẽ đóng vai trò là một trình tạo câu trả lời sai (hallucinated answer generator). Với một câu hỏi, câu trả lời đúng và kiến thức liên quan, mục tiêu của bạn là viết một câu trả lời nghe có vẻ hợp lý nhưng thực tế lại sai.
Bạn PHẢI tạo ra câu trả lời sai bằng cách sau: 
Sai thông tin (Factual Contradiction): Câu trả lời có mâu thuẫn thực tế với kiến thức được cung cấp. Bạn có thể bịa ra một số thông tin không có trong dữ liệu gốc.
Bạn PHẢI ghi rõ nhãn loại sai **trước câu trả lời** theo định dạng sau:  
**[Sai thông tin]: Câu trả lời bịa đặt...**  
Chỉ trả lời theo yêu cầu mà không nhắc lại hướng dẫn hoặc lưu ý gì khác."""

instruction_2 = """Bạn sẽ đóng vai trò là một trình tạo câu trả lời sai (hallucinated answer generator). Với một câu hỏi, câu trả lời đúng và kiến thức liên quan, mục tiêu của bạn là viết một câu trả lời nghe có vẻ hợp lý nhưng thực tế lại sai.
Bạn PHẢI tạo ra câu trả lời sai bằng cách sau:
Quá chung hoặc quá cụ thể (Specificity Error): Câu trả lời quá chung chung hoặc quá chi tiết so với mức độ phù hợp của câu hỏi.
Bạn PHẢI ghi rõ nhãn loại sai **trước câu trả lời** theo định dạng sau:  
**[Quá chung hoặc quá cụ thể]: Câu trả lời bịa đặt...**  
Chỉ trả lời theo yêu cầu mà không nhắc lại hướng dẫn hoặc lưu ý gì khác."""

instruction_3 = """Bạn sẽ đóng vai trò là một trình tạo câu trả lời sai (hallucinated answer generator). Với một câu hỏi, câu trả lời đúng và kiến thức liên quan, mục tiêu của bạn là viết một câu trả lời nghe có vẻ hợp lý nhưng thực tế lại sai.
Bạn PHẢI tạo ra câu trả lời sai bằng cách sau:
Lập luận sai (Incorrect Reasoning): Bạn suy luận từ kiến thức nhưng đi đến một kết luận sai, không thể suy ra từ dữ liệu ban đầu.
Bạn PHẢI ghi rõ nhãn loại sai **trước câu trả lời** theo định dạng sau:  
**[Lập luận sai]: Câu trả lời bịa đặt...**  
Chỉ trả lời theo yêu cầu mà không nhắc lại hướng dẫn hoặc lưu ý gì khác."""

In [7]:
def get_qa_res(topic, question, answer, instruction):
    if isinstance(instruction, str):
        message = [
            {"role": "system", "content": instruction},  
            {"role": 
             "user", "content":
                                "\n\n#Chủ đề#: " + topic +
                                "\n#Câu hỏi#: " + question +
                                "\n#Câu trả lời đúng#: " + answer +
                                "\n#Câu trả lời bịa đặt#: "}
        ]
    else:
        raise TypeError("The instruction must be str or list!")
             
    while True:
        try:
            res = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=message,
                temperature=1,
                max_tokens=4000,
                top_p=1
            )
            break
        except openai.error.RateLimitError:
            print('openai.error.RateLimitError\nRetrying...')
            time.sleep(60)
        except openai.error.ServiceUnavailableError:
            print('openai.error.ServiceUnavailableError\nRetrying...')
            time.sleep(20)
        except openai.error.Timeout:
            print('openai.error.Timeout\nRetrying...')
            time.sleep(20)
        except openai.error.APIError:
            print('openai.error.APIError\nRetrying...')
            time.sleep(20)
        except openai.error.APIConnectionError:
            print('openai.error.APIConnectionError\nRetrying...')
            time.sleep(20)
    
    
    return res.choices[0].message.content


In [14]:
new_data = {"Chủ đề": [], "Câu hỏi": [], "Câu trả lời đúng": [], "Câu trả lời bịa đặt": []}
def generate_qa_dataset(data, instruction_0, instruction_1, instruction_2, instruction_3):
    for i in range(6000,data.shape[0]):

        question = data.iloc[i]['Text']
        answer = data.iloc[i]['Câu trả lời']
        topic = data.iloc[i]['Ministry']
        if i%4 == 0:
            ans = get_qa_res(topic, question, answer, instruction_0)
        elif i%4 == 1:
            ans = get_qa_res(topic, question, answer, instruction_1)
        elif i%4 == 2:
            ans = get_qa_res(topic, question, answer, instruction_2)
        else:
            ans = get_qa_res(topic, question, answer, instruction_3)
        new_data["Chủ đề"].append(topic)
        new_data["Câu hỏi"].append(question)
        new_data["Câu trả lời đúng"].append(answer)
        new_data["Câu trả lời bịa đặt"].append(ans)
        print(" sample {} completed!".format(i))

In [15]:
generate_qa_dataset(data, instruction_0, instruction_1, instruction_2, instruction_3)


 sample 6000 completed!
 sample 6001 completed!
 sample 6002 completed!
 sample 6003 completed!
 sample 6004 completed!
 sample 6005 completed!
 sample 6006 completed!
 sample 6007 completed!
 sample 6008 completed!
 sample 6009 completed!
 sample 6010 completed!
 sample 6011 completed!
 sample 6012 completed!
 sample 6013 completed!
 sample 6014 completed!
 sample 6015 completed!
 sample 6016 completed!
 sample 6017 completed!
 sample 6018 completed!
 sample 6019 completed!
 sample 6020 completed!
 sample 6021 completed!
 sample 6022 completed!
 sample 6023 completed!
 sample 6024 completed!
 sample 6025 completed!
 sample 6026 completed!
 sample 6027 completed!
 sample 6028 completed!
 sample 6029 completed!
 sample 6030 completed!
 sample 6031 completed!
 sample 6032 completed!
 sample 6033 completed!
 sample 6034 completed!
 sample 6035 completed!
 sample 6036 completed!
 sample 6037 completed!
 sample 6038 completed!
 sample 6039 completed!
 sample 6040 completed!
 sample 6041 com

In [16]:
new_data_df = pd.DataFrame(new_data)
new_data_df.to_csv("Generated Data 7.csv", index=False, encoding='utf-8-sig')

In [12]:
import json

In [17]:

#tach new_data thanh json co key la cac cot
json_data = json.dumps([dict(zip(new_data.keys(), values)) for values in zip(*new_data.values())], ensure_ascii=False, indent=4)
with open('Generated Data 7.json', 'w', encoding='utf-8') as f:
    f.write(json_data)


In [ ]:
# gop tat ca cac file 1-> 7 json lai thanh 1 generate_data.json va csv lại thanh generate_data.csv
#import os
#import json
#import pandas as pd

#data = []
#for file in os.listdir():
    #if file.startswith('Generated Data') and file.endswith('.json'):
        #with open(file, 'r', encoding='utf-8') as f:
            #data.extend(json.load(f))

#with open('Generated_data.json', 'w', encoding='utf-8') as f:
    #json.dump(data, f, ensure_ascii=False, indent=4)

#df = pd.DataFrame(data)
#df.to_csv('Generated_data.csv', index=False, encoding='utf-8-sig')

In [ ]:
#import tiktoken

Caculate max answer token 

In [ ]:
#encoding = tiktoken.encoding_for_model("gpt-4o-mini")

In [ ]:
#max_token_answer = 0
#indexmax = 0
#for i in range(data.shape[0]):
    #answer_token = encoding.encode(data.iloc[i]['Câu trả lời'])
    #max_token_answer = max(max_token_answer, len(answer_token))
    #indexmax = i
    #if max_token_answer == len(answer_token):
        #print("Max token answer: ", max_token_answer)
        #print("Index: ", indexmax)
#max_token_answer

    